### Este notebook presenta un proceso automatizado para navegar en la página web y extraer datos sobre vuelos dentro de los Estados Unidos desde el sitio web: https://www.transtats.bts.gov/ONTIME/Departures.aspx


In [8]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

Primero, almacenamos la URL de la página web de vuelos de la que extraeremos la información. Luego, creamos una función diseñada para desplegar y extraer las opciones disponibles. La función realiza una solicitud GET a la URL, analiza el HTML de la página y extrae las opciones del menú.

Una vez creada la función, la utilizamos dos veces: primero para obtener la lista de aeropuertos y luego para obtener la lista de aerolíneas. Estas listas nos permitirán, más adelante, iterar sobre ellas en un bucle, seleccionando para cada aeropuerto todas las aerolíneas disponibles.

In [9]:
url = "https://www.transtats.bts.gov/ONTIME/Departures.aspx"

def obtener_opciones(url, aeropuertos_aerolineas):
    """Con esta función se seleccionan los aeropuertos o las aerolíneas que están en un desplegable"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    seleccionar_aeropuerto_aerolinea = soup.find("select", attrs={"name": aeropuertos_aerolineas})
    opciones = seleccionar_aeropuerto_aerolinea.find_all("option")

    listado_opciones = [opcion.text for opcion in opciones]

    return listado_opciones

listado_aeropuertos = obtener_opciones(url, "cboAirport")
listado_aerolineas  = obtener_opciones(url, "cboAirline")

Esta función automatiza la selección de elementos en un menú desplegable. Se utiliza específicamente para seleccionar los aeropuertos y aerolíneas previamente almacenados en dos listas. La elección de By.NAME se debe al hecho de que estos elementos están identificados por su nombre en el código HTML.

In [5]:
def seleccionar_por_indice(url, nombre_seleccionado, lista_elementos):
    """Esta función selecciona los aeropuertos y las aerolineas en el menú desplegable"""
    driver = webdriver.Firefox()
    driver.get(url)

    seleccion = Select(driver.find_element(By.NAME, nombre_seleccionado))
    
    for elemento in lista_elementos:
        seleccion.select_by_visible_text(elemento)
        sleep(1) 

La función preselecciones se utiliza al inicio para marcar de antemano todas las estadísticas (por qué sale tarde, el tiempo que tarda en despegar...), todos los días del mes, el mes de diciembre y los tres años con los que hemos hecho el trabajo. 

In [6]:
def preselecciones(driver):
    """Preselecciona las casillas necesarias para la extracción"""
    driver.find_element(By.ID, "chkAllStatistics").click() 
    driver.find_element(By.ID, "chkAllDays").click()  
    driver.find_element(By.ID, "chkMonths_11").click()     # click_mes_diciembre
    
    # Selecciona 2021, 2022 y 2023
    driver.find_element(By.ID, "chkYears_34").click()
    driver.find_element(By.ID, "chkYears_35").click() 
    driver.find_element(By.ID, "chkYears_36").click() 

Este script automatiza el proceso de extracción de datos de la página web para cada combinación de aeropuertos y aerolíneas

- Iniciamos el navegador Firefox usando Selenium y se carga la URL especificada.
- Llamamos a la función preselecciones para marcar las opciones de estadísticas, días, meses y años antes de comenzar la extracción.
- Se inicia un bucle que itera sobre cada aeropuerto en la lista listado_aeropuertos.
- Se inicia otro bucle dentro del bucle anterior, que itera sobre cada aerolínea en la lista listado_aerolineas.
- Se desplaza la página hacia abajo para asegurar que se pueda hacer clic en el enlace de descarga de los .CSV.
- Si ese aeropuerto con esa aerolínea no opera, se pasa.
- Cuando termina cierra el navegador

In [9]:
driver = webdriver.Firefox()
driver.get(url)

preselecciones(driver)

for aeropuerto in listado_aeropuertos:

    select_aeropuerto = Select(driver.find_element(By.NAME, "cboAirport"))
    select_aeropuerto.select_by_visible_text(aeropuerto)

    # sleep(1)
    for aerolinea in listado_aerolineas:

        select_aerolinea = Select(driver.find_element(By.NAME, "cboAirline"))
        select_aerolinea.select_by_visible_text(aerolinea)
    
        click_submit = driver.find_element(By.ID, "btnSubmit").click()
        
        # sleep(1)
        #el codigo de abajo hace una flecha hacia abajo,
        driver.execute_script("window.scrollBy(0, 200);")

        try:
            element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "DL_CSV")))
            # sleep(1)
            element.click()
            
        except:
            pass

driver.quit()